In [1]:
import pygame
import random
import time
import csv

width = 600
height = 400
Display = pygame.display.set_mode((width, height))
pygame.display.set_caption("단어 맞추기 게임")
pygame.mixer.init()
pygame.font.init()

font = pygame.font.SysFont("malgungothic", 30)

def load_words(filename):
    with open(filename, "r") as f:
        words = [line.strip() for line in f]
    return words

def random_words(words, num_words):
    return random.sample(words, num_words)

words = load_words("data/word.txt")

# 게임 설정
input_setup_active = True
num_words_input_str = ""
time_limit_input_str = ""
user_num_words = 10 
user_time_limit = 5

setup_stage = "num_words" 

while input_setup_active:
    Display.fill((255, 255, 255))

    if setup_stage == "num_words":
        instruction_text = font.render("문제 수 (1 ~ 20)", True, (0, 0, 0))
        input_box_text = font.render(num_words_input_str, True, (0, 0, 0))
    elif setup_stage == "time_limit":
        instruction_text = font.render("시간 제한 (초)", True, (0, 0, 0))
        input_box_text = font.render(time_limit_input_str, True, (0, 0, 0))

    instruction_rect = instruction_text.get_rect(center=(width // 2, height // 3))
    Display.blit(instruction_text, instruction_rect)
    input_box_rect = input_box_text.get_rect(center=(width // 2, height // 2))
    Display.blit(input_box_text, input_box_rect)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            game_running = False 
            input_setup_active = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                if setup_stage == "num_words":
                    try:
                        user_num_words = int(num_words_input_str)
                        if 1 <= user_num_words <= 20: # 문제 수 유효성 검사
                            setup_stage = "time_limit"
                        else:
                            num_words_input_str = "" # 유효하지 않으면 다시 입력 요청
                    except ValueError:
                        num_words_input_str = "" # 숫자가 아니면 다시 입력 요청
                elif setup_stage == "time_limit":
                    try:
                        user_time_limit = float(time_limit_input_str)
                        if user_time_limit <= 0:
                            user_time_limit = 5 # 유효하지 않으면 기본값으로
                        input_setup_active = False # 시간 제한 설정 완료
                    except ValueError:
                        time_limit_input_str = "" # 유효하지 않으면 다시 입력 요청
            elif event.key == pygame.K_BACKSPACE:
                if setup_stage == "num_words":
                    num_words_input_str = num_words_input_str[:-1]
                elif setup_stage == "time_limit":
                    time_limit_input_str = time_limit_input_str[:-1]
            elif event.unicode.isdigit() or (setup_stage == "time_limit" and event.unicode == '.'):
                if setup_stage == "num_words":
                    num_words_input_str += event.unicode
                elif setup_stage == "time_limit":
                    time_limit_input_str += event.unicode

    pygame.display.flip()

    if not input_setup_active and not hasattr(locals(), 'game_running'):
        game_running = True


# --- 사용자 설정으로 게임 초기화 ---
target_words = random_words(words, user_num_words)
user_inputs = [""] * user_num_words
time_limit = user_time_limit
word_positions = [None] * user_num_words
current_word_index = 0
correct_count = 0
game_finish = False
input_active = True
start_time = time.time()
total_start_time = time.time()


# --- 게임 루프 ---
while game_running:
    current_time = time.time()
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            game_running = False
        elif event.type == pygame.KEYDOWN and not game_finish and input_active:
            if event.key == pygame.K_RETURN:
                if user_inputs[current_word_index] == target_words[current_word_index]:
                    correct_count += 1
                    pygame.mixer.music.load('assets/good.wav')
                    pygame.mixer.music.play()
                else:
                    pygame.mixer.music.load("assets/bad.wav")
                    pygame.mixer.music.play()
                current_word_index += 1
                if current_word_index >= user_num_words:
                    game_finish = True
                    input_active = False
                else:
                    start_time = time.time()
            elif event.key == pygame.K_BACKSPACE:
                user_inputs[current_word_index] = user_inputs[current_word_index][:-1]
            elif event.unicode.isalpha() or event.unicode.isprintable():
                user_inputs[current_word_index] += event.unicode

    Display.fill((255, 255, 255))

    if current_word_index < user_num_words:
        if current_time - start_time > time_limit:
            pygame.mixer.music.load("assets/bad.wav")
            pygame.mixer.music.play()
            current_word_index += 1
            if current_word_index >= user_num_words:
                game_finish = True
                input_active = False
            else:
                start_time = time.time()
        else:
            if word_positions[current_word_index] is None:
                random_x = random.randint(100, 400)
                random_y = random.randint(50, 300)
                word_positions[current_word_index] = (random_x, random_y)

            text = font.render(f"{target_words[current_word_index]}", True, (0, 0, 0))
            Display.blit(text, word_positions[current_word_index])

            input_text = font.render(f"정답 {current_word_index + 1}: {user_inputs[current_word_index]}", True, (0, 0, 0))
            input_rect = input_text.get_rect(center=(width // 2, height * 9 // 10))
            Display.blit(input_text, input_rect)

    if game_finish:
        end_time = time.time()
        game_time = end_time - total_start_time
        result_text = font.render(f"맞춘 개수: {correct_count} / {user_num_words}", True, (0, 0, 0))
        Display.blit(result_text, (200, 350))
        pass_criteria = int(user_num_words * 0.7)
        if correct_count >= pass_criteria:
            final_result_text = font.render("합격입니다!", True, (0, 0, 0))
        else:
            final_result_text = font.render("불합격입니다!", True, (0, 0, 0))
        final_result_rect = final_result_text.get_rect(center=(width // 2, height // 2))
        Display.blit(final_result_text, final_result_rect)

        with open('word_game_score.csv', 'w', newline='', encoding='utf-8') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['게임 시간', '맞춘 개수', '문제 수', '시간 제한'])
            writer.writerow([game_time, correct_count, user_num_words, user_time_limit])

    pygame.display.flip()

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.8)
Hello from the pygame community. https://www.pygame.org/contribute.html
